In [1]:
import os
from utils import DataSeq, load_ids

import numpy as np
import pandas as pd
import skimage.io as io

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout

import keras_tuner as kt

from imblearn.over_sampling import RandomOverSampler

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

2023-08-30 11:08:49.981989: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-30 11:08:50.172036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 11:08:51.049285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
path = "./Train_non_experts_simple/"
batch_size = 8
epochs = 10

In [3]:
ids, labels = load_ids(path)
print(ids.shape, labels.shape)
X_train, X_valid, y_train, y_valid = train_test_split(ids, labels, test_size=0.2, random_state=42)

2023-08-30 11:08:53.677111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 11:08:53.786481: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 11:08:53.786532: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 11:08:53.789572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 11:08:53.789625: I tensorflow/compile

(75243,) (75243, 3)


ead NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-30 11:08:54.824883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


# Oversampling the data

In [4]:
over_sampler = RandomOverSampler()
X_train_over, y_train_over = over_sampler.fit_resample(X_train.reshape(-1, 1), y_train)
X_train_over = X_train_over.flatten()

# Check if labels are correct
all_equal = True
for i in range(100):
    # Compare path label vs y_train_over label
    if np.argmax(y_train_over[i]) != int(X_train_over[i][27]) - 1:
        all_equal = False

print(all_equal)

True


In [5]:
values, counts = np.unique(np.argmax(y_train,axis=1), return_counts = True)
print("Training labels BEFORE oversampling")

for val, count in zip(values, counts):
    print(val, count)

y = np.argmax(y_train_over,axis=1)
values, counts = np.unique(y, return_counts = True)
print("\nTraining labels AFTER oversampling")

for val, count in zip(values, counts):
    print(val, count)

Training labels BEFORE oversampling
0 29793
1 22193
2 8208

Training labels AFTER oversampling
0 29793
1 29793
2 29793


In [6]:
train_seq = DataSeq(X_train_over, y_train_over)
valid_seq = DataSeq(X_valid, y_valid)

train_steps = len(X_train_over) // batch_size
valid_steps = len(X_valid) // batch_size

# Hyperparameter Tuning

In [7]:
 def build_transfer_vgg_aug(hp):
    input_layer = Input(shape=(224, 224, 3))

    data_augmentation = tf.keras.Sequential([
        input_layer,
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.15),
    ])
    
    vgg = tf.keras.applications.VGG16(weights=None, include_top=False, input_tensor=data_augmentation.output)
    vgg.load_weights('./vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
    vgg.trainable = False

    model = vgg.output
    model = Flatten()(model)

    neurons = hp.Choice("neurons", [1024, 2048, 4096])

    model = Dense(neurons, activation='relu')(model)
    model = Dropout(0.2)(model)
    model = Dense(neurons, activation='relu')(model)
    model = Dropout(0.2)(model)
    model = Dense(3, activation='softmax')(model)

    model = Model(input_layer, model, name="Transfer_VGG16")
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [8]:
obj = kt.Objective("val_accuracy", "max")
# tuner = kt.Hyperband(build_transfer_vgg_aug,
#                      objective=obj,
#                      max_epochs=epochs,
#                      factor=3,
#                      seed = 42,
#                      directory='./tuner_results/',
#                      project_name='BCSS')

tuner = kt.RandomSearch(
    hypermodel=build_transfer_vgg_aug,
    objective=obj,
    max_trials=3,
    executions_per_trial=1,
    overwrite=True,
    directory='./tuner_results/',
    project_name='BCSS',
)

tuner.search_space_summary()

Search space summary
Default search space size: 1
neurons (Choice)
{'default': 1024, 'conditions': [], 'values': [1024, 2048, 4096], 'ordered': True}


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

tuner.search(train_seq, validation_data=valid_seq, steps_per_epoch=train_steps,
   validation_steps=valid_steps, epochs=epochs, callbacks=[stop_early])

Trial 2 Complete [00h 52m 48s]
val_accuracy: 0.8312067985534668

Best val_accuracy So Far: 0.8344630599021912
Total elapsed time: 02h 05m 03s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
1024              |2048              |neurons

Epoch 1/10
11172/11172 [==============================] - 504s 45ms/step - loss: 0.5337 - accuracy: 0.7892 - val_loss: 0.4372 - val_accuracy: 0.8373
Epoch 2/10
11171/11172 [============================>.] - ETA: 0s - loss: 0.4681 - accuracy: 0.8133

In [12]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"The optimal number of neurons per layer is {best_hps.get('neurons')}")

The optimal number of neurons per layer is 1024


In [16]:
model = tuner.hypermodel.build(best_hps)
model.summary()

Model: "Transfer_VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 random_flip_3 (RandomFlip)  (None, 224, 224, 3)       0         
                                                                 
 random_rotation_3 (RandomR  (None, 224, 224, 3)       0         
 otation)                                                        
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                    

In [18]:
tuner.results_summary()

Results summary
Results in ./tuner_results/BCSS
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
neurons: 1024
Score: 0.8373205661773682

Trial 0 summary
Hyperparameters:
neurons: 2048
Score: 0.8344630599021912

Trial 1 summary
Hyperparameters:
neurons: 4096
Score: 0.8312067985534668


In [ ]:
history = model.fit(train_seq, validation_data=valid_seq, steps_per_epoch=train_steps,
 validation_steps=valid_steps, epochs=epochs, callbacks=[stop_early])

Epoch 1/10
 6045/11172 [===============>..............] - ETA: 3:21 - loss: 0.5691 - accuracy: 0.7794